In [ ]:
import chromadb
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
# LangChain v0.2.x에서는 'from langchain.chains import ConversationalRetrievalChain' 대신
# 'langchain.chains.retrieval_qa'를 사용하고 memory를 직접 주입하는 방식이 더 흔합니다.

def run_rag_chatbot(keyword):
    """
    RAG 챗봇을 실행하고 사용자의 질문에 답변합니다. (메모리 기능 추가)
    """
    try:
        # 1. 벡터 데이터베이스 불러오기
        db_path = f"./chroma_db_{keyword}"
        client = chromadb.PersistentClient(path=db_path)
        
        collection_name = f"{keyword}_news_collection"
        
        # 2. 임베딩 모델 로드
        embeddings_model = SentenceTransformerEmbeddings(model_name="jhgan/ko-sroberta-multitask")
        
        # 3. LangChain의 Chroma 클래스로 벡터스토어 로드
        vectorstore = Chroma(
            client=client,
            collection_name=collection_name,
            embedding_function=embeddings_model
        )

        # 4. LangChain 리트리버 설정
        retriever = vectorstore.as_retriever(
            search_type="similarity", 
            search_kwargs={"k": 3} 
        )
        
        # 5. Ollama를 통해 로컬 LLM 로드
        llm = Ollama(model="llama3")

        # 6. 대화 메모리 추가
        # 이전 대화 내용을 저장하여 LLM에게 전달하는 역할
        memory = ConversationBufferMemory(
            memory_key="chat_history", 
            return_messages=True
        )

        # 7. RetrievalQA 체인 구성 (메모리 추가)
        # 이 부분이 수정되었습니다.
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            memory=memory # 메모리 객체를 체인에 주입
        )

        print("-" * 50)
        print("챗봇이 준비되었습니다. '종료'를 입력하면 대화가 종료됩니다.")
        print("-" * 50)

        while True:
            question = input("질문: ").strip()
            if question.lower() == "종료":
                print("챗봇을 종료합니다.")
                break
            
            # 8. 질문에 대한 답변 생성 및 출력
            # 이제 qa_chain이 대화 기록을 자동으로 관리합니다.
            response = qa_chain.invoke({"query": question})
            print("챗봇: " + response['result'])
            print("-" * 50)

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
        print("Ollama가 정상적으로 실행 중인지 확인하세요. (ollama run llama3)")

# 실행 코드
if __name__ == "__main__":
    search_keyword = input("챗봇에 사용할 데이터의 키워드(영문)를 입력하세요: ").strip()
    run_rag_chatbot(search_keyword)